# CK0223 - Mineração de Dados

## Lista 05 - Regressão

### Dados do discente:
**Nome**: Luiza Esther Martins Pessoa
**Matrícula**: 555516

### Vídeo Youtube:
[Mineração de Dados: Lista 05 - Regressão (Explicando o código)]()

### GitHub:
[EstherMart - Regressão](https://github.com/EstherMart/Data-Mining/blob/main/Lista04_AnaliseExploratoria/analise_exploratoria.ipynb)

---

### **(a)** Ler o dataset *fakeTelegram.BR_2022.csv*

Ler o dataset é o primeiro passo para iniciarmos a extração, manipulação e tratameto dos dados, para que, depois disso, seja possível realizar a regressão.

Portanto, começamos importando as bibliotecas necessárias para leitura (`pandas`) e para download local da base de dados (`gdown`). Como já dito anteriormente durante a resolução das listas passadas, existem outras formas de realizar o upload para o repositório local, mas decidi seguir a lógica de puxar e realizar o download de base utilizando apenas o link disponibilizado pelo professor.

**Importante**: Trechos de código serão reutilizados, visto que são as mesmas exigências em ambas as listas e tal solicitação já foi realizada nas listas anteriores.

In [ ]:
# IMPORTAÇÃO DE BIBLIOTECAS
import gdown
import pandas as pd
from IPython.display import display

O código abaixo é o mesmo desenvolvido anteriormente. Ele utiliza `gdown` para baixar a base de dados no repositório local.

In [ ]:
origem_url = 'https://drive.google.com/file/d/1c_hLzk85pYw-huHSnFYZM_gn-dUsYRDm/view'

# O ID do arquivo (necessário para fazer o download direto) está entre os últimos elementos da URL.
# Fazemos um split na URL usando '/' como separador e pegamos o penúltimo elemento da lista.
# Isso funciona porque a estrutura da URL é:
# https://drive.google.com/file/d/ID_DO_ARQUIVO/view
# E ao aplicar url.split('/'), o resultado será:
# ['https:', '', 'drive.google.com', 'file', 'd', 'ID_DO_ARQUIVO', 'view?...']
# Portanto, o ID está na posição -2 (penúltima).

file_id = origem_url.split('/')[-2]

# URL do arquivo no formato aceito pelo gdown
url = f'https://drive.google.com/uc?id={file_id}'

# Nome local do arquivo que será baixado
output = 'fakeTelegram.BR_2022.csv'

# Baixando o arquivo com gdown
gdown.download(url, output, quiet=False)

Leitura do dataset utilizando o método `.read_csv()`, permitindo carregar dados estruturados a partir de arquivos *CSV* para um DataFrame do pandas.

In [2]:
df_inicial = pd.read_csv("fakeTelegram.BR_2022.csv")

Para fins de **visualização e verificação inicial dos dados carregados**, o código abaixo é usado para:
- Obter uma compreensão rápida da dimensão do dataset
- Identificar os nomes das colunas disponíveis
- Realizar uma primeira checagem da integridade básica da estrutura de dados

In [3]:
# PARA MELHOR VISUALIZAÇÃO
print("Número de linhas:", df_inicial.shape[0])
print("Número de colunas:", df_inicial.shape[1])
print("\nColunas disponíveis:")
for i, col in enumerate(df_inicial.columns, 1):
    print(f"{i}. {col}")

# VISUALIZANDO AMOSTRAS INICIAIS E FINAIS DO DATASET
display(df_inicial.head().style.set_caption("Primeiros Registros").set_properties(**{
    'background-color': '#f8f9fa',
    'border': '1px solid #dee2e6',
    'color': '#212529',
    'max-width': '300px',
    'overflow': 'hidden',
    'text-overflow': 'ellipsis',
    'white-space': 'nowrap'
}))

display(df_inicial.tail().style.set_caption("Últimos Registros").set_properties(**{
    'background-color': '#f8f9fa',
    'border': '1px solid #dee2e6',
    'color': '#212529',
    'max-width': '300px',
    'overflow': 'hidden',
    'text-overflow': 'ellipsis',
    'white-space': 'nowrap'
}))

Número de linhas: 557586
Número de colunas: 19

Colunas disponíveis:
1. date_message
2. id_member_anonymous
3. id_group_anonymous
4. media
5. media_type
6. media_url
7. has_media
8. has_media_url
9. trava_zap
10. text_content_anonymous
11. dataset_info_id
12. date_system
13. score_sentiment
14. score_misinformation
15. id_message
16. message_type
17. messenger
18. media_name
19. media_md5


,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,dataset_info_id,date_system,score_sentiment,score_misinformation,id_message,message_type,messenger,media_name,media_md5
0,2022-10-05 06:25:04,1078cc958f0febe28f4d03207660715f,12283e08a2eb5789201e105b34489ee7,nan,nan,nan,False,False,False,Então é Fato Renato o áudio que eu ouvi no whatsapp isso ocorreu em Niterói principalmente no bairro Fonseca ?,5,2022-10-05 06:25:28.863641,0.000000,nan,16385,Texto,telegram,nan,nan
1,2022-10-05 06:25:08,nan,12283e08a2eb5789201e105b34489ee7,nan,nan,nan,False,False,False,"Saiu no YouTube do presidente a 8 horas atrás, infelizmente não consigo enviar para cá, mas é facilmente verificável no YouTube do presidente",5,2022-10-05 06:25:28.926311,0.064400,nan,16386,Texto,telegram,nan,nan
2,2022-10-05 06:26:28,92a2d8fd7144074f659d1d29dc3751da,9f2d7394334eb224c061c9740b5748fc,nan,nan,nan,False,False,False,"É isso, nossa parte já foi quase toda feita. No segundo turno completamos nossa parte desse teatro. Essa é uma guerra de 4* geração na dimensão humana e uma guerra espiritual do bem contra o mal na dimensão do Universo. Pensamento positivo é fundamental, pensem sempre em algo bom. Deus continua nos abençoando, nosso livre arbítrio completa o curso.",5,2022-10-05 06:26:29.361949,-0.355100,0.157242,16366,Texto,telegram,nan,nan
3,2022-10-05 06:27:28,d60aa38f62b4977426b70944af4aff72,c8f2de56550ed0bf85249608b7ead93d,94dca4cda503100ebfda7ce2bcc060eb.jpg,image/jpg,nan,True,False,False,GENTE ACHEI ELES EM UMA SEITA MAÇONÁRICA,5,2022-10-05 06:27:29.935624,0.000000,nan,19281,Imagem,telegram,nan,94dca4cda503100ebfda7ce2bcc060eb
4,2022-10-05 06:27:44,cd6979b0b5265f08468fa1689b6300ce,e56ec342fc599ebb4ed89655eb6f03aa,5ad5c8bbe9da93a37fecf3e5aa5b0637.jpg,image/jpg,nan,True,False,False,nan,5,2022-10-05 06:28:29.316325,nan,nan,507185,Imagem,telegram,nan,5ad5c8bbe9da93a37fecf3e5aa5b0637


,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,dataset_info_id,date_system,score_sentiment,score_misinformation,id_message,message_type,messenger,media_name,media_md5
557581,2022-11-11 12:06:15,333e9869f23dbd4682d1be382d9c1e59,e56ec342fc599ebb4ed89655eb6f03aa,25e43b6a58b848c43ad5b5f9e979822a.jpg,url,https://terrabrasilnoticias.com/2022/11/bndes-tem-lucro-de-r-96-bilhoes-no-terceiro-trimestre/,True,True,False,"BNDES tem lucro de R$ 9,6 bilhões no terceiro trimestre ☛ https://terrabrasilnoticias.com/2022/11/bndes-tem-lucro-de-r-96-bilhoes-no-terceiro-trimestre/",5,2022-11-16 14:49:39.146502,0.102700,nan,575796,Url,telegram,nan,25e43b6a58b848c43ad5b5f9e979822a
557582,2022-11-11 12:09:08,nan,5b10d7739171149be6d9961e3350c071,657949d03e4088f6b332e2686ccd3221.jpg,url,https://youtu.be/8g1Vz9_0xVk,True,True,False,https://youtu.be/8g1Vz9_0xVk,5,2022-11-16 14:49:39.847434,0.000000,nan,1286443,Url,telegram,nan,657949d03e4088f6b332e2686ccd3221
557583,2022-11-11 12:09:47,nan,1590a03f43b5ba4b6147a1c5e1dd357b,a21848a61045380a6483866daed0ca0e.jpg,image/jpg,https://t.me/vemprasruas,True,True,False,"Empresários, demitam os petistas primeiro. https://t.me/vemprasruas",5,2022-11-16 14:49:39.922279,0.000000,nan,13294,Imagem,telegram,nan,a21848a61045380a6483866daed0ca0e
557584,2022-11-11 12:09:46,nan,5b10d7739171149be6d9961e3350c071,a21848a61045380a6483866daed0ca0e.jpg,image/jpg,https://t.me/vemprasruas,True,True,False,"Empresários, demitam os petistas primeiro. https://t.me/vemprasruas",5,2022-11-16 14:49:39.992932,0.000000,nan,1286444,Imagem,telegram,nan,a21848a61045380a6483866daed0ca0e
557585,2022-11-11 12:09:48,nan,b11f2df64ac19aad47a50accf32052d6,a21848a61045380a6483866daed0ca0e.jpg,image/jpg,https://t.me/vemprasruas,True,True,False,"Empresários, demitam os petistas primeiro. https://t.me/vemprasruas",5,2022-11-16 14:49:40.064006,0.000000,nan,192127,Imagem,telegram,nan,a21848a61045380a6483866daed0ca0e


---

### **(b)** Removendo os traza-zaps, linhas repetidas (duplicada) e texto com menos de 5 palavras

In [ ]:
df_inicial.dtypes

#### Limpeza Inicial dos Dados
- **Remoção de trava-zaps**: Utilizei a coluna `trava_zap`, que já marca mensagens com esse tipo de comportamento.
- **Remoção de duplicatas**: Mensagens com o mesmo conteúdo textual foram removidas.
- **Filtragem por tamanho do texto**: Criei uma nova coluna `qtd_palavras` e removi mensagens com menos de 5 palavras.

#### Análise das Linhas Removidas

Antes de aplicar cada etapa de limpeza, armazenei as linhas que seriam removidas para análise posterior. Isso permite:

- Verificar o impacto de cada etapa;
- Validar que nenhuma informação relevante foi descartada indevidamente;
- Documentar o processo de forma clara e reprodutível.

In [4]:
# Cópia original preservada
df_original = df_inicial.copy()

# ------------------------------
# 1. Linhas marcadas como trava-zap
# ------------------------------
trava_zaps_removidos = df_original[df_original['trava_zap']].copy()
print(f"Trava-zaps removidos: {trava_zaps_removidos.shape[0]} linhas")
display(trava_zaps_removidos.head().style.set_caption("Exemplo de Trava-Zaps Removidos"))

df_sem_travazap = df_original[~df_original['trava_zap']].copy()


Trava-zaps removidos: 16 linhas


,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,dataset_info_id,date_system,score_sentiment,score_misinformation,id_message,message_type,messenger,media_name,media_md5
21944,2022-10-07 07:46:52,nan,c712c1b704c22bd0cef50bc06125cdbd,nan,nan,nan,False,False,True,nan,5,2022-10-07 07:47:00.355052,0.000000,0.067344,53260,Texto,telegram,nan,nan
89109,2022-10-16 00:45:02,8a30ac374bc4b5930eaf0667a178546a,e56ec342fc599ebb4ed89655eb6f03aa,nan,nan,nan,False,False,True,nan,5,2022-10-16 00:45:04.064662,0.000000,0.056698,521324,Texto,telegram,nan,nan
294541,2022-10-04 14:22:47,39ee10516124280a22f1798f2a41f9a7,959f13e0079883060632c74ffc81c547,nan,nan,nan,False,False,True,nan,5,2022-10-04 14:22:48.808572,0.973400,0.010433,27241,Texto,telegram,nan,nan
324567,2022-10-25 14:55:55,e003fbb6ffedb1838e42360d41cab314,5b10d7739171149be6d9961e3350c071,nan,nan,nan,False,False,True,nan,5,2022-10-25 14:56:11.604972,0.946000,0.403945,1182938,Texto,telegram,nan,nan
389164,2022-10-30 20:19:52,nan,c8f2de56550ed0bf85249608b7ead93d,nan,nan,nan,False,False,True,nan,5,2022-10-30 20:19:54.183578,0.000000,nan,28330,Texto,telegram,nan,nan


In [5]:
# ------------------------------
# 2. Linhas duplicadas por texto
# ------------------------------
duplicadas_removidas = df_sem_travazap[df_sem_travazap.duplicated(keep='first')].copy()
print(f"Linhas completamente duplicadas removidas: {duplicadas_removidas.shape[0]}")
display(duplicadas_removidas.head().style.set_caption("Exemplos de Duplicatas Completas Removidas"))

df_sem_duplicadas = df_sem_travazap.drop_duplicates().copy()

Linhas completamente duplicadas removidas: 0


,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,dataset_info_id,date_system,score_sentiment,score_misinformation,id_message,message_type,messenger,media_name,media_md5


In [6]:
# ------------------------------
# 3. Linhas com menos de 5 palavras
# ------------------------------
df_sem_duplicadas['qtd_palavras'] = df_sem_duplicadas['text_content_anonymous'].apply(lambda x: len(str(x).split()))
linhas_curta_removidas = df_sem_duplicadas[df_sem_duplicadas['qtd_palavras'] < 5].copy()
print(f"Mensagens com menos de 5 palavras removidas: {linhas_curta_removidas.shape[0]} linhas")
display(linhas_curta_removidas.head().style.set_caption("Exemplo de Textos Muito Curtos Removidos"))

Mensagens com menos de 5 palavras removidas: 218284 linhas


,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,dataset_info_id,date_system,score_sentiment,score_misinformation,id_message,message_type,messenger,media_name,media_md5,qtd_palavras
4,2022-10-05 06:27:44,cd6979b0b5265f08468fa1689b6300ce,e56ec342fc599ebb4ed89655eb6f03aa,5ad5c8bbe9da93a37fecf3e5aa5b0637.jpg,image/jpg,nan,True,False,False,nan,5,2022-10-05 06:28:29.316325,nan,nan,507185,Imagem,telegram,nan,5ad5c8bbe9da93a37fecf3e5aa5b0637,1
7,2022-10-05 06:43:51,a7db4ff6a6d35e57be5bcf02f400cec6,857cd5311da1bdc15eb9e6918a47c6c6,4fe93a3772445f64173ef03db1fed83c.jpg,image/jpg,nan,True,False,False,nan,5,2022-10-05 06:43:54.51408,nan,nan,891535,Imagem,telegram,nan,4fe93a3772445f64173ef03db1fed83c,1
8,2022-10-05 07:09:39,d2c33afd13a3165be031b6c244f7140d,d9c080b9db8cd9d0e88e870ce782c01a,30f29cca618753961cb97be95e87e3b4.jpg,image/jpg,nan,True,False,False,nan,5,2022-10-05 07:10:16.569761,nan,nan,314861,Imagem,telegram,nan,30f29cca618753961cb97be95e87e3b4,1
15,2022-10-05 06:35:31,32a71ef400256cdd21f5b02e1d1afd19,0e899038fbe9196d404c90cf7ad282e0,019140562d645bfe6ffd923f609b89ac.jpg,image/jpg,nan,True,False,False,nan,5,2022-10-05 06:35:49.09209,nan,nan,5755,Imagem,telegram,nan,019140562d645bfe6ffd923f609b89ac,1
18,2022-10-05 06:37:09,047b67065d87ac7d8db977ad4506b436,c8f2de56550ed0bf85249608b7ead93d,nan,nan,https://www.facebook.com/groups/864827680253646/permalink/5431674063568962/?flite=scwspnss&mibextid=7EG2rCgC370kf71V,False,True,False,https://www.facebook.com/groups/864827680253646/permalink/5431674063568962/?flite=scwspnss&mibextid=7EG2rCgC370kf71V,5,2022-10-05 06:37:29.584697,0.000000,nan,19284,Texto,telegram,nan,nan,1


In [9]:
# Exibir amostra de dados pós-limpeza
df_limpo = df_sem_duplicadas[df_sem_duplicadas['qtd_palavras'] >= 5].copy()
print(f"\nDataset final após todas as limpezas: {df_limpo.shape[0]} linhas e {df_limpo.shape[1]} colunas")
display(df_limpo.head(5).style.set_caption("Amostra Após Limpeza").set_properties(**{
    'background-color': '#f8f9fa',
    'border': '1px solid #dee2e6',
    'color': '#212529',
    'max-width': '400px',
    'overflow': 'hidden',
    'text-overflow': 'ellipsis',
    'white-space': 'nowrap'
}))


Dataset final após todas as limpezas: 339286 linhas e 20 colunas


,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,dataset_info_id,date_system,score_sentiment,score_misinformation,id_message,message_type,messenger,media_name,media_md5,qtd_palavras
0,2022-10-05 06:25:04,1078cc958f0febe28f4d03207660715f,12283e08a2eb5789201e105b34489ee7,nan,nan,nan,False,False,False,Então é Fato Renato o áudio que eu ouvi no whatsapp isso ocorreu em Niterói principalmente no bairro Fonseca ?,5,2022-10-05 06:25:28.863641,0.000000,nan,16385,Texto,telegram,nan,nan,20
1,2022-10-05 06:25:08,nan,12283e08a2eb5789201e105b34489ee7,nan,nan,nan,False,False,False,"Saiu no YouTube do presidente a 8 horas atrás, infelizmente não consigo enviar para cá, mas é facilmente verificável no YouTube do presidente",5,2022-10-05 06:25:28.926311,0.064400,nan,16386,Texto,telegram,nan,nan,23
2,2022-10-05 06:26:28,92a2d8fd7144074f659d1d29dc3751da,9f2d7394334eb224c061c9740b5748fc,nan,nan,nan,False,False,False,"É isso, nossa parte já foi quase toda feita. No segundo turno completamos nossa parte desse teatro. Essa é uma guerra de 4* geração na dimensão humana e uma guerra espiritual do bem contra o mal na dimensão do Universo. Pensamento positivo é fundamental, pensem sempre em algo bom. Deus continua nos abençoando, nosso livre arbítrio completa o curso.",5,2022-10-05 06:26:29.361949,-0.355100,0.157242,16366,Texto,telegram,nan,nan,59
3,2022-10-05 06:27:28,d60aa38f62b4977426b70944af4aff72,c8f2de56550ed0bf85249608b7ead93d,94dca4cda503100ebfda7ce2bcc060eb.jpg,image/jpg,nan,True,False,False,GENTE ACHEI ELES EM UMA SEITA MAÇONÁRICA,5,2022-10-05 06:27:29.935624,0.000000,nan,19281,Imagem,telegram,nan,94dca4cda503100ebfda7ce2bcc060eb,7
5,2022-10-05 06:28:30,nan,b52442a5fbc459ae590dca0d215e32f9,nan,nan,nan,False,False,False,Kķkkkkk to rindo até agora....Quem disse q ia fazer acordo até com o diabo nao foi o presidente Bolsonaro e sim a estoca vento 😂😂😂😂😂😂,5,2022-10-05 06:29:29.046694,0.700300,0.197813,2735,Texto,telegram,nan,nan,25


---

### **(c)** Agrupamento de linhas com postagens iguais ou extremamente semelhantes. 

Depois de realizar a limpeza inicial do dataframe (**df_limpo**), decidi aplicar um pré-processamento textual robusto nos dados da coluna `text_content_anonymous`, que contém o conteúdo das mensagens. O objetivo é preparar os textos para tarefas como agrupamento, análise semântica ou modelagem, removendo ruídos e padronizando o conteúdo.

- Remoção de valores nulos: Eliminei registros onde o texto está ausente, pois não fazem sentido para análise textual.
- Normalização para minúsculas: Converto todo o texto para letras minúsculas para evitar distinções artificiais entre palavras como "Brasil" e "brasil".
- Normalização de URLs: Substituo URLs completas por apenas o domínio. Isso ajuda a identificar menções a sites como "youtube", "facebook", etc., sem manter links completos e únicos.
- Padronização de emojis repetidos: Quando emojis aparecem repetidamente (ex: "😂😂😂😂"), preservo a quantidade mas separo por espaço para que cada emoji seja tratado individualmente.
- Redução de caracteres repetidos: Reduzo repetições excessivas de caracteres (ex: "loooool" → "lool"), o que suaviza exageros mas ainda mantém a intenção do autor.
- Remoção de símbolos não informativos: Elimino tudo o que não for letra, número ou os símbolos que ainda quero manter, respeitando a linguagem informal das mensagens.
- Remoção de stopwords: Uso a lista de stopwords do NLTK em português, complementada com gírias/comuns da internet que não agregam semântica (ex: "vc", "pra", "tá").
- Tokenização e filtragem de palavras curtas: Elimino palavras com 2 letras ou menos, pois geralmente são ruído.
- Remoção de acentuação: Faço a normalização de caracteres acentuados para a forma ASCII, o que reduz a complexidade do vocabulário (ex: "ação" → "acao").

O resultado final é uma nova coluna chamada `texto_pre_processado` no meu df_limpo.

In [20]:
import re
import unicodedata
import pandas as pd
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

# Remove textos nulos
df_limpo = df_limpo.dropna(subset=["text_content_anonymous"])

# Define função de pré-processamento
def preprocess_text(text):
    if pd.isna(text):
        return ""
    
    text = str(text).lower()
    
    # Normaliza URLs, mantendo apenas o domínio
    text = re.sub(r'https?://(?:www\.)?([a-zA-Z0-9.-]+)(?:/.*)?', r'\1', text)
    
    # Reduz emojis repetidos mantendo separação
    def replace_repeated_emojis(match):
        emoji = match.group(1)
        count = match.group(0).count(emoji)
        return ' '.join([emoji] * count)
    text = re.sub(r'((:[a-zA-Z_]+:)(?:\2)+)', replace_repeated_emojis, text)
    
    # Reduz repetições excessivas de caracteres (ex: loooool → lool)
    text = re.sub(r'(.)\1{3,}', r'\1\1\1', text)
    
    # Remove símbolos indesejados, mantendo alguns específicos
    text = re.sub(r'[^\w\s*_:áéíóúâêîôûãõàèìòù]', '', text)
    
    # Remove stopwords e palavras curtas
    stop_words = set(stopwords.words('portuguese'))
    extra_stopwords = {'q', 'pra', 'vc', 'vcs', 'tá', 'tbm', 'tb', 'hj', 'pq', 'to', 'ta', 'tô'}
    stop_words.update(extra_stopwords)
    
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words and len(word) > 2]
    text = ' '.join(filtered_words)
    
    # Remove acentuação
    text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII')
    
    return text

# Aplica o pré-processamento
df_limpo['texto_pre_processado'] = df_limpo['text_content_anonymous'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\esthe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Garante que o conteúdo das colunas seja exibido por completo
pd.set_option('display.max_colwidth', None)

# Visualiza 5 exemplos aleatórios antes e depois do pré-processamento
display(
    df_limpo[['text_content_anonymous', 'texto_pre_processado']].sample(5, random_state=42)
)

,text_content_anonymous,texto_pre_processado
413567,This community was blocked in Brazil following a decision of the Superior Electoral Court (TSE).,this community was blocked brazil following decision the superior electoral court tse
261023,"Gente boa tarde, tudo bem?\nAcabei de votar, votei na Anhembi Morumbi na rua casa do ator, quando digitei 22, a máquina deu tipo uma travada e depois digitei novamente 22 e confirmei aí que foi....achei estranho sabe...",gente boa tarde tudo bem acabei votar votei anhembi morumbi rua casa ator digitei maquina deu tipo travada digitei novamente confirmei foiachei estranho sabe
65969,Essa é a esquerda sempre passando pano pra pedófilos 🤢🤢,esquerda sempre passando pano pedofilos
379455,"Eu sou da Bahia e faço parte da menoria da direita. Nunca fui de esquerda. Aqui, os da direita também tem sido massacrados e perseguidos pelos esquerdistas. Lamentável e infelizmente essa gama que votaram em Lula e Geronimo para governador não são apenas os pobres, miseráveis do bolsa Brasil. Infelizmente tem os ricos que se aplanam com o prefeitos corruptos do interior do Estado e que pagam por votos com cestas básicas... Infelizmente essa é a realidade. Enquanto houver esse ""social de preguiçoso que mantém os necessitados, haverá essa gama de escravos.",bahia faco parte menoria direita nunca esquerda aqui direita sido massacrados perseguidos esquerdistas lamentavel infelizmente gama votaram lula geronimo governador apenas pobres miseraveis bolsa brasil infelizmente ricos aplanam prefeitos corruptos interior estado pagam votos cestas basicas infelizmente realidade enquanto social preguicoso mantem necessitados havera gama escravos
84769,"Welcome, Rafael\n\n🔸 [USER] — professional tool for managing Telegram groups",welcome rafael user professional tool managing telegram groups


Com o texto das mensagens já pré-processado, o próximo passo é agrupar postagens iguais ou muito semelhantes. Isso ajuda a identificar conteúdos replicados, reenviados ou viralizados entre grupos.

Para isso, usei a técnica de **MinHash com Locality-Sensitive Hashing (LSH)**, que permite encontrar rapidamente textos semelhantes em grandes volumes de dados de forma eficiente.

1. Criei uma função para gerar MinHash: ela transforma cada texto em uma "assinatura" hash baseada nas palavras que ele contém.

2. Configurei o LSH com um threshold de similaridade alto (0.9) para garantir que só mensagens realmente parecidas fossem agrupadas.

3. Comparei cada mensagem com as demais: para cada índice, recuperei todos os textos semelhantes e os agrupei, evitando repetição com um set() de visitados.

4. Atribuí um ID de grupo (group_id) para cada conjunto de mensagens semelhantes.

5. Criei a coluna `qtd_compartilhamentos`, que representa o número de vezes que a mesma (ou quase a mesma) mensagem apareceu.

6. Mantive apenas uma mensagem por grupo: aquela com a data mais antiga, garantindo que a origem cronológica seja respeitada.

O resultado é um dataframe consolidado por similaridade textual, ideal para análises de propagação de mensagens.

In [24]:
from datasketch import MinHash, MinHashLSH
from tqdm import tqdm

tqdm.pandas()  # para barra de progresso com apply()

# Função para gerar MinHash de um texto
def create_minhash(text, num_perm=128):
    mh = MinHash(num_perm=num_perm)
    for word in text.split():
        mh.update(word.encode('utf-8'))
    return mh

# Criar LSH
lsh = MinHashLSH(threshold=0.9, num_perm=128)
minhashes = {}

print("🔄 Gerando MinHashes e inserindo no LSH...")

for idx, text in tqdm(df_limpo['texto_pre_processado'].items(), total=len(df_limpo)):
    mh = create_minhash(text)
    lsh.insert(idx, mh)
    minhashes[idx] = mh

print("🔍 Agrupando mensagens semelhantes...")

groups = []
visited = set()

for idx in df_limpo.index:
    if idx not in visited:
        similar = lsh.query(minhashes[idx])
        visited.update(similar)
        groups.append(similar)

print(f"✅ Total de grupos encontrados: {len(groups)}")

# Mapear cada índice para seu grupo
group_map = {}
for group_id, members in enumerate(groups):
    for idx in members:
        group_map[idx] = group_id

df_limpo['group_id'] = df_limpo.index.map(group_map)

# Criar coluna com quantidade de compartilhamentos por grupo
df_limpo['qtd_compartilhamentos'] = df_limpo['group_id'].map(df_limpo['group_id'].value_counts())

# Pegar a linha mais antiga de cada grupo
df_limpo['date_message'] = pd.to_datetime(df_limpo['date_message'])
df_agrupado = df_limpo.sort_values('date_message').drop_duplicates(subset='group_id', keep='first')

# Resetar índice
df_agrupado = df_agrupado.reset_index(drop=True)

🔄 Gerando MinHashes e inserindo no LSH...


100%|██████████| 339286/339286 [12:06<00:00, 467.07it/s]  


🔍 Agrupando mensagens semelhantes...
✅ Total de grupos encontrados: 203617


In [25]:
print("\n📊 Exemplo de dados agrupados:")
display(df_agrupado[['text_content_anonymous', 'texto_pre_processado', 'group_id', 'qtd_compartilhamentos', 'date_message']].head())

print(f"\n📌 Linhas após agrupamento: {df_agrupado.shape[0]}")
print(f"📌 Colunas mantidas: {df_agrupado.shape[1]}")


📊 Exemplo de dados agrupados:


,text_content_anonymous,texto_pre_processado,group_id,qtd_compartilhamentos,date_message
0,Bitcoin has really changed my life for the best... I didn't believe when I was seeing Alot of people complaining of been scammed I was scared of been scammed too till This company convinced me and I took a try of $400 for their promo package and I got $4000 after a day of trading with them \nI'm really happy\nJOIN THE WINNING TEAM\nJOIN THE WINNING TEAM\nJOIN THE WINNING TEAM,bitcoin has really changed life the best didnt believe when was seeing alot people complaining been scammed was scared been scammed too till this company convinced and took try 400 their promo package and got 4000 after day trading with them really happy join the winning team join the winning team join the winning team,73487,1,2022-09-29 00:00:04
1,"é, mas o circo tá armado pra fazer aquele espetáculo pastelão.\nEu queria era ver o circo pegar fogo pra acabar com isso, e não escapar um palhaço",circo armado fazer espetaculo pastelao queria ver circo pegar fogo acabar escapar palhaco,73488,1,2022-09-29 00:00:07
2,"BOE TOMOU MEDIDAS DE EMERGÊNCIA NA QUARTA-FEIRA PARA EVITAR UM DERRAME NO SETOR DE PENSÃO DO REINO UNIDO, QUE PODERIA TER SIDO O 'MOMENTO LEHMAN' DO REINO UNIDO: FT\n\nFonte: Távola Redonda Financeira\n\n🔸 t.me/alexeconomia",boe tomou medidas emergencia quartafeira evitar derrame setor pensao reino unido poderia ter sido momento lehman reino unido: fonte: tavola redonda financeira tmealexeconomia,73489,1,2022-09-29 00:00:11
3,Caramba! Por isso o meu não compartilha nem todos os vídeos. FDP SAFADOS! Vou desinstalar e baixar o novo!,caramba compartilha todos videos fdp safados vou desinstalar baixar novo,73490,1,2022-09-29 00:00:13
4,"Welcome, Suely Pimentel\n\n🔸 [USER] — professional tool for managing Telegram groups",welcome suely pimentel user professional tool managing telegram groups,73491,1,2022-09-29 00:00:15



📌 Linhas após agrupamento: 203617
📌 Colunas mantidas: 24
